In [2]:
import psycopg2
import pandas as pd
import numpy as np

In [3]:
# NEVER share passwords

dbname = "dev"
host = "redshift-cluster-1.c9gt5btzchps.eu-central-1.redshift.amazonaws.com"
port = 5439
user = ""
password = ""

In [4]:
# connect to redshift

connect = psycopg2.connect(
        dbname=dbname, host=host, port=port, user=user, password=password
    )

cursor = connect.cursor()

In [6]:
# query all rows and columns from online_transactions table

query = """select *
           from bootcamp.stock_description
           limit 10
           """

pd.read_sql(query, connect)

/var/folders/gs/c69h38_15hb4fnkpnc9gvksm0000gn/T/ipykernel_86715/2569024018.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, connect)


,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE
5,10124G,ARMY CAMO BOOKCOVER TAPE
6,10125,MINI FUNKY DESIGN TAPES
7,10133,COLOURING PENCILS BROWN TUBE
8,10135,COLOURING PENCILS BROWN TUBE
9,11001,ASSTD DESIGN RACING CAR PEN


In [7]:
# query all rows and columns from online_transactions table

query = """select *
           from bootcamp.online_transactions
           """

online_transactions = pd.read_sql(query, connect)
online_transactions

/var/folders/gs/c69h38_15hb4fnkpnc9gvksm0000gn/T/ipykernel_86715/808454499.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  online_transactions = pd.read_sql(query, connect)


,invoice,stock_code,quantity,invoice_date,price,customer_id,country
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom
1,536365,84029E,6,2010-12-01 08:26:00,3.39,u1785,United Kingdom
2,536366,22632,6,2010-12-01 08:28:00,1.85,u1785,United Kingdom
3,536368,22914,3,2010-12-01 08:34:00,4.95,u13047,United Kingdom
4,536367,22749,8,2010-12-01 08:34:00,3.75,u13047,United Kingdom
...,...,...,...,...,...,...,...
541905,581586,21217,24,2011-12-09 12:49:00,8.95,u13113,United Kingdom
541906,581587,22555,12,2011-12-09 12:50:00,1.65,u1268,France
541907,581587,22730,4,2011-12-09 12:50:00,3.75,u1268,France
541908,581587,22613,12,2011-12-09 12:50:00,0.85,u1268,France


In [8]:
# query all rows and columns from stock_description table

query = """select *
           from bootcamp.stock_description
           """

stock_description = pd.read_sql(query, connect)
stock_description


/var/folders/gs/c69h38_15hb4fnkpnc9gvksm0000gn/T/ipykernel_86715/2430299846.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stock_description = pd.read_sql(query, connect)


,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE
...,...,...
3947,DCGSSGIRL,GIRLS PARTY BAG
3948,DOT,DOTCOM POSTAGE
3949,PADS,PADS TO MATCH ALL CUSHIONS
3950,POST,POSTAGE


In [42]:
query = """select count(*)
           from bootcamp.online_transactions
           where customer_id = ''
           """

pd.read_sql(query, connect)


/var/folders/gs/c69h38_15hb4fnkpnc9gvksm0000gn/T/ipykernel_86715/502995770.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, connect)


,count
0,135080


In [9]:
type(online_transactions)

pandas.core.frame.DataFrame

In [10]:
type(stock_description)

pandas.core.frame.DataFrame

In [11]:
online_transactions.shape

# 54k rows and 7 columns

(541910, 7)

In [12]:
stock_description.shape

# 4k rows and 2 columns

(3952, 2)

In [13]:
# check for duplicated data

online_transactions.duplicated().sum()

# 5270 rows of duplicated data

5270

In [14]:
# preview the duplicated data

online_transactions[online_transactions.duplicated()]

# we can either drop all the duplicated data or keep only one

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
294,536464,22866,1,2010-12-01 12:23:00,2.10,u17968,United Kingdom
972,536637,22867,1,2010-12-02 11:35:00,2.10,u18041,United Kingdom
1236,536796,22585,1,2010-12-02 15:46:00,1.25,u15574,United Kingdom
1251,536796,22902,1,2010-12-02 15:46:00,2.10,u15574,United Kingdom
1451,536863,22758,1,2010-12-03 11:19:00,1.25,u17967,United Kingdom
...,...,...,...,...,...,...,...
541412,581450,22118,1,2011-12-08 17:54:00,1.25,u16794,United Kingdom
541440,581456,35964,2,2011-12-08 18:42:00,0.39,u1753,United Kingdom
541843,581538,22694,1,2011-12-09 11:34:00,2.10,u14446,United Kingdom
541846,581538,23318,1,2011-12-09 11:34:00,2.49,u14446,United Kingdom


In [15]:
# an example of duplicated data
# both rows are identical

online_transactions[(online_transactions.invoice == "536464") &
                   (online_transactions.stock_code == "21992")]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
11485,536464,21992,1,2010-12-01 12:23:00,2.95,u17968,United Kingdom
11496,536464,21992,1,2010-12-01 12:23:00,2.95,u17968,United Kingdom


In [16]:
# using describe to look at the distribution of data

online_transactions.describe()

,quantity,price
count,541910.000000,541910.000000
mean,9.552234,4.611138
std,218.080957,96.759765
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [17]:
# using describe to look at the distribution of data

stock_description.describe()

# hmm.. Iulia spotted there were 47 cases of ? in description, let's investigate

,stock_code,description
count,3952,3952
unique,3905,3785
top,22600,?
freq,2,47


In [18]:
# oh no... we have ?, and 6 stock codes with the same description and other duplicates appearing. This is such a mess!
# we may need to recalculate the stock code

stock_description.description.value_counts()

description
?                                     47
METAL SIGN,CUPCAKE SINGLE HOOK         6
CINAMMON SET OF 9 T-LIGHTS             2
COLUMBIAN CANDLE RECTANGLE             2
3 WHITE CHOC MORRIS BOXED CANDLES      2
                                      ..
MAGNETS PACK OF 4 SWALLOWS             1
MAGNETS PACK OF 4 CHILDHOOD MEMORY     1
MAGNETS PACK OF 4 HOME SWEET HOME      1
MAGNETS PACK OF 4 VINTAGE COLLAGE      1
SAMPLES                                1
Name: count, Length: 3785, dtype: int64

In [19]:
# stock codes where description is ?

stock_description[stock_description.description == "?"]

,stock_code,description
36,16020C,?
64,16207B,?
390,21145,?
451,21232,?
535,21368,?
584,21427,?
594,21446,?
675,21591,?
865,21877,?
997,22077,?


In [20]:
# invoices where price is less than 0.. oh! we see we have missing customer ids in this case

online_transactions[online_transactions.price < 0]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
304933,A563186,B,1,2011-08-12 14:51:00,-11062.06,,United Kingdom
350910,A563187,B,1,2011-08-12 14:52:00,-11062.06,,United Kingdom


In [21]:
# invoices where quantity is less than 0

online_transactions[online_transactions.quantity < 0]

# we may want to remove rows of data where quantity < 0 in future iterations

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
59,C536391,21984,-24,2010-12-01 10:24:00,0.29,u17548,United Kingdom
60,C536391,22557,-12,2010-12-01 10:24:00,1.65,u17548,United Kingdom
138,C536391,22556,-12,2010-12-01 10:24:00,1.65,u17548,United Kingdom
139,C536391,21484,-12,2010-12-01 10:24:00,3.45,u17548,United Kingdom
572,C536548,22244,-4,2010-12-01 14:33:00,1.95,u12472,Germany
...,...,...,...,...,...,...,...
541452,C581463,85048,-2,2011-12-08 18:56:00,7.95,u17526,United Kingdom
541453,C581465,22171,-1,2011-12-08 18:59:00,8.50,u15755,United Kingdom
541454,C581466,22720,-2,2011-12-08 19:20:00,4.95,u13883,United Kingdom
541544,C581490,22178,-12,2011-12-09 09:57:00,1.95,u14397,United Kingdom


In [22]:
# where one invoice has a negative quantity less than 0

online_transactions.quantity.min()

-80995

In [23]:
# look for missing data in both tables

online_transactions.isnull().sum()

# whoa, what is going on here. we just saw some missing customer ids. Looks like
# when migrating the data to redshift, the missing data was not handled correctly

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
dtype: int64

In [24]:
# but I can try to fix this by replacing the blanks with python recognised nan

online_transactions = online_transactions.replace('', np.nan)                   # to get rid of empty values

# we will fix this is in our transformation tasks

In [25]:
# check for missing values again

online_transactions.isnull().sum()

# 135k missing customer ids!!!! We will drop invoices with missing customer ids in our transformation tasks

invoice              0
stock_code           0
quantity             0
invoice_date         0
price                0
customer_id     135080
country              0
dtype: int64

In [26]:
# percentage of customers without an id

round(online_transactions.customer_id.isnull().sum()/len(online_transactions.customer_id.unique()))

31

In [27]:
# look for missing data in stock description table

stock_description.isnull().sum()

# hmm...

stock_code     0
description    0
dtype: int64

In [28]:
# fix missing data

stock_description = stock_description.replace('', np.nan)   
stock_description.isnull().sum()

# ok, cant see any missing data right now, but looks like someone replaced the missing values with ?

stock_code     0
description    0
dtype: int64

In [29]:
# check for the summary of both tables

online_transactions.info()

# invoice_date is appearing as data type object and not date time, everything else looks good

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       541910 non-null  object 
 1   stock_code    541910 non-null  object 
 2   quantity      541910 non-null  int64  
 3   invoice_date  541910 non-null  object 
 4   price         541910 non-null  float64
 5   customer_id   406830 non-null  object 
 6   country       541910 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 28.9+ MB


In [30]:
stock_description.info()

# loos good, but will double check how much stock codes are in the online_transactions table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3952 entries, 0 to 3951
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   stock_code   3952 non-null   object
 1   description  3952 non-null   object
dtypes: object(2)
memory usage: 61.9+ KB


In [31]:
len(online_transactions.stock_code.unique())

# hmm.. why does online_transactions have more stock codes?

4070

In [39]:
# writing a query to identify how many stock codes have been purchased but are not in the stock description database

query = """select count(distinct t1.stock_code)
           from bootcamp.online_transactions t1
           left join (select *
                      from bootcamp.stock_description
                      where description <> '?') t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
           """

pd.read_sql(query, connect)



/var/folders/gs/c69h38_15hb4fnkpnc9gvksm0000gn/T/ipykernel_86715/2934140866.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, connect)


,count
0,165


In [40]:
# looking at the data where stock codes are missing in the stock description table

query = """select *
           from bootcamp.online_transactions t1
           left join (select *
                      from bootcamp.stock_description
                      where description <> '?') t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
           """

pd.read_sql(query, connect)

# one observation, there is a missing customer id so will filter out the missing customer_id

/var/folders/gs/c69h38_15hb4fnkpnc9gvksm0000gn/T/ipykernel_86715/1986491744.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, connect)


,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536544,None,1,2010-12-01 14:32:00,1.66,,United Kingdom,None
1,536549,None,1,2010-12-01 14:34:00,0.00,,United Kingdom,None
2,536557,None,1,2010-12-01 14:41:00,1.45,u17841,United Kingdom,None
3,536569,None,1,2010-12-01 15:35:00,1.25,u16274,United Kingdom,None
4,536592,None,1,2010-12-01 17:06:00,4.21,,United Kingdom,None
...,...,...,...,...,...,...,...,...
2374,581217,None,1,2011-12-08 09:20:00,0.83,,United Kingdom,None
2375,581336,None,1,2011-12-08 12:10:00,15.00,u16161,United Kingdom,None
2376,581424,None,4,2011-12-08 15:29:00,1.45,u1389,United Kingdom,None
2377,581439,None,3,2011-12-08 16:30:00,0.83,,United Kingdom,None


In [38]:
# looking at the data where stock codes are missing in the stock description table

query = """select count(distinct t1.stock_code)
           from bootcamp.online_transactions t1
           left join (select *
                      from bootcamp.stock_description
                      where description <> '?') t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
            and customer_id <> ''
           """

pd.read_sql(query, connect)

# one observation, there is a missing customer id so will filter out the missing customer_id
# only 22 stock codes 

/var/folders/gs/c69h38_15hb4fnkpnc9gvksm0000gn/T/ipykernel_86715/2075819427.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, connect)


,count
0,22


In [37]:
# looking at the data where stock codes are missing in the stock description table

query = """select distinct t1.stock_code
           from bootcamp.online_transactions t1
           left join (select *
                      from bootcamp.stock_description
                      where description <> '?') t2 on t1.stock_code = t2.stock_code
           where t2.stock_code is null
            and customer_id <> ''
           """

pd.read_sql(query, connect)

# one observation, there is a missing customer id so will filter out the missing customer_id
# only 22 stock codes 

/var/folders/gs/c69h38_15hb4fnkpnc9gvksm0000gn/T/ipykernel_86715/526138726.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, connect)


,stock_code
0,46000R
1,M
2,46000U
3,20964
4,46000M
5,BANK CHARGES
6,18007
7,23444
8,16162M
9,21705


In [7]:
connect.close()

Data Quality Audit
- We have 5270 rows of duplicated data and we keep one instance
- 31% of customers did not have customer id
- We need to fix the invoice_date field
- We need to do further investigation on stock code, but 22 stock codes without customer_id do not have a descriptions.
- We have 10k invoices with a negative quantity, with one invoice having a negative quantity of 81k. Thanks Faith!
- We have spotted 47 examples of the description having a ?, which we will replace with Unknown. Thanks Iulia!
